<a href="https://colab.research.google.com/github/Rumeysakeskin/Question-Answering-BERT/blob/main/bert-squad-question-answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-wt26tafp/nemo-toolkit_d830fdc11b714e87804164ac9ff982fb
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-wt26tafp/nemo-toolkit_d830fdc11b714e87804164ac9ff982fb
  Resolved https://github.com/NVIDIA/NeMo.git to commit d2add6f870a0368ec6880d73cc45b0bd73c891a7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [18]:
import os
import wget
import gc

import pytorch_lightning as pl
from omegaconf import OmegaConf

from nemo.collections.nlp.models.question_answering.qa_bert_model import BERTQAModel
from nemo.utils.exp_manager import exp_manager

pl.seed_everything(42)
gc.disable()

INFO:lightning_fabric.utilities.seed:Global seed set to 42


In [19]:
# set the following paths
DATA_DIR = "data_dir" # directory for storing datasets
WORK_DIR = "work_dir" # directory for storing trained models, logs, additionally downloaded scripts

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(WORK_DIR, exist_ok=True)

In [20]:
# download the model's default configuration file 
config_dir = WORK_DIR + '/conf/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + "qa_conf.yaml"):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/conf/qa_conf.yaml', config_dir)
else:
    print ('config file already exists')

# this will print the entire default config of the model
config_path = f'{WORK_DIR}/conf/qa_conf.yaml'
print(config_path)
config = OmegaConf.load(config_path)
print("Default Config - \n")
print(OmegaConf.to_yaml(config))

config file already exists
work_dir/conf/qa_conf.yaml
Default Config - 

pretrained_model: null
do_training: true
trainer:
  devices:
  - 0
  num_nodes: 1
  max_epochs: 3
  max_steps: -1
  accumulate_grad_batches: 1
  gradient_clip_val: 1.0
  precision: 16
  accelerator: gpu
  log_every_n_steps: 5
  val_check_interval: 1.0
  resume_from_checkpoint: null
  num_sanity_val_steps: 0
  enable_checkpointing: false
  logger: false
  strategy: ddp
model:
  tensor_model_parallel_size: 1
  nemo_path: null
  library: huggingface
  save_model: false
  tokens_to_generate: 32
  dataset:
    version_2_with_negative: true
    doc_stride: 128
    max_query_length: 64
    max_seq_length: 512
    max_answer_length: 30
    use_cache: false
    do_lower_case: true
    check_if_answer_in_context: true
    keep_doc_spans: all
    null_score_diff_threshold: 0.0
    n_best_size: 20
    num_workers: 1
    pin_memory: false
    drop_last: false
  train_ds:
    file: null
    batch_size: 24
    shuffle: true
    

In [21]:
# download get_squad.py script to download and preprocess the SQuAD data
os.makedirs(WORK_DIR, exist_ok=True)
if not os.path.exists(WORK_DIR + '/get_squad.py'):
    print('Downloading get_squad.py...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/question_answering/get_squad.py', WORK_DIR)
else:
    print ('get_squad.py already exists')

get_squad.py already exists


In [22]:
# download and preprocess the data
!python $WORK_DIR/get_squad.py --destDir $DATA_DIR
!ls -LR {DATA_DIR}/squad

[NeMo I 2023-04-10 10:13:23 get_squad:66] data_dir
[NeMo I 2023-04-10 10:13:23 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
[NeMo I 2023-04-10 10:13:23 get_squad:49] ** Download file already exists, skipping download
[NeMo I 2023-04-10 10:13:23 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
[NeMo I 2023-04-10 10:13:23 get_squad:49] ** Download file already exists, skipping download
[NeMo I 2023-04-10 10:13:23 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
[NeMo I 2023-04-10 10:13:23 get_squad:49] ** Download file already exists, skipping download
[NeMo I 2023-04-10 10:13:23 get_squad:47] Downloading: https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
[NeMo I 2023-04-10 10:13:23 get_squad:49] ** Download file already exists, skipping download
data_dir/squad:
v1.1  v2.0

data_dir/squad/v1.1:
dev-v1.1.json  train-v1.1.json

data_dir/squ

In [23]:
# if True, model will load features from cache if file is present, or
# create features and dump to cache file if not already present
config.model.dataset.use_cache = False

# indicates whether the dataset has unanswerable questions
config.model.dataset.version_2_with_negative = True

# indicates whether the dataset is of extractive nature or not
# if True, context spans/chunks that do not contain answer are treated as unanswerable 
config.model.dataset.check_if_answer_in_context = True

# set file paths for train, validation, and test datasets
config.model.train_ds.file = f"{DATA_DIR}/squad/v2.0/train-v2.0.json"
config.model.validation_ds.file = f"{DATA_DIR}/squad/v2.0/dev-v2.0.json"
config.model.test_ds.file = f"{DATA_DIR}/squad/v2.0/dev-v2.0.json"

# set batch sizes for train, validation, and test datasets
config.model.train_ds.batch_size = 8
config.model.validation_ds.batch_size = 8
config.model.test_ds.batch_size = 8

# set number of samples to be used from dataset. setting to -1 uses entire dataset
config.model.train_ds.num_samples = 5000
config.model.validation_ds.num_samples = 1000
config.model.test_ds.num_samples = 100

In [24]:
config.trainer.max_epochs = 1
config.trainer.max_steps = -1 # takes precedence over max_epochs
config.trainer.precision = 16
config.trainer.devices = [0] # 0 for CPU, or list of the GPUs to use [0] this tutorial does not support multiple GPUs. If needed please use NeMo/examples/nlp/question_answering/question_answering.py
config.trainer.accelerator = "gpu"
config.trainer.strategy="dp"

In [25]:
config.exp_manager.exp_dir = WORK_DIR
config.exp_manager.name = "QA-SQuAD2"
config.exp_manager.create_wandb_logger=False

In [26]:
# set language model and tokenizer to be used
# tokenizer is derived from model if a tokenizer name is not provided
config.model.language_model.pretrained_model_name = "bert-base-uncased"
config.model.tokenizer.tokenizer_name = "bert-base-uncased"

# path where model will be saved
config.model.nemo_path = f"{WORK_DIR}/checkpoints/bert_squad_v2_0.nemo"

config.exp_manager.create_checkpoint_callback = True

config.model.optim.lr = 3e-5

In [27]:
trainer = pl.Trainer(**config.trainer)
model = BERTQAModel(config.model, trainer=trainer)
trainer.fit(model)
trainer.test(model)

model.save_to(config.model.nemo_path)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


[NeMo I 2023-04-10 10:13:24 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: None, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-04-10 10:13:25 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2023-04-10 10:13:25 qa_processing:106] mean no. of chars in doc: 839.2727272727273
[NeMo I 2023-04-10 10:13:25 qa_processing:107] max no. of chars in doc: 1895
[NeMo I 2023-04-10 10:13:25 qa_processing:106] mean no. of chars in doc: 677.5487804878048
[NeMo I 2023-04-10 10:13:25 qa_processing:107] max no. of chars in doc: 1782
[NeMo I 2023-04-10 10:13:25 qa_processing:106] mean no. of chars in doc: 828.0972222222222
[NeMo I 2023-04-10 10:13:25 qa_processing:107] max no. of chars in doc: 2132
[NeMo I 2023-04-10 10:13:25 qa_processing:106] mean no. of chars in doc: 540.0
[NeMo I 2023-04-10 10:13:25 qa_processing:107] max no. of chars in doc: 1423
[NeMo I 2023-04-10 10:13:25 qa_processing:106] mean no. of chars in doc: 756.71875
[NeMo I 2023-04-10 10:13:25 qa_processing:107] max no. of chars in doc: 1747
[NeMo I 2023-04-10 10:13:25 qa_processing:106] mean no. of chars in doc: 732.4418604651163
[NeMo I 2023-04-10 10:13:25 qa_processing:107] max no. of chars in doc: 3076
[NeMo I 2023

  0%|          | 0/5000 [00:00<?, ?it/s]

...

[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val exact: 29.1
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val f1: 34.41956674569356
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val total: 1000.0
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val HasAns_exact: 58.032128514056225
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val HasAns_f1: 68.71398944918384
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val HasAns_total: 498.0
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val NoAns_exact: 0.398406374501992
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val NoAns_f1: 0.398406374501992
[NeMo I 2023-04-10 10:17:43 qa_bert_model:140] val NoAns_total: 502.0


INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test exact: 30.0
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test f1: 33.999633699633705
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test total: 100.0
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test HasAns_exact: 66.66666666666667
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test HasAns_f1: 75.55474155474157
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test HasAns_total: 45.0
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test NoAns_exact: 0.0
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test NoAns_f1: 0.0
[NeMo I 2023-04-10 10:17:47 qa_bert_model:140] test NoAns_total: 55.0


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test_HasAns_exact     │     66.66666412353516     │
│      test_HasAns_f1       │     75.55474090576172     │
│     test_HasAns_total     │           45.0            │
│     test_NoAns_exact      │            0.0            │
│       test_NoAns_f1       │            0.0            │
│     test_NoAns_total      │           55.0            │
│        test_exact         │           30.0            │
│          test_f1          │     33.9996337890625      │
│         test_loss         │     5.231266021728516     │
│        test_total         │           100.0           │
└───────────────────────────┴───────────────────────────┘

In [28]:
model = BERTQAModel.restore_from(config.model.nemo_path)

eval_device = [config.trainer.devices[0]] if isinstance(config.trainer.devices, list) else 1
model.trainer = pl.Trainer(
    devices=eval_device,
    accelerator=config.trainer.accelerator,
    precision=16,
    logger=False,
)

config.exp_manager.create_checkpoint_callback = False
exp_dir = exp_manager(model.trainer, config.exp_manager)
output_nbest_file = os.path.join(exp_dir, "output_nbest_file.json")
output_prediction_file = os.path.join(exp_dir, "output_prediction_file.json")

all_preds, all_nbest = model.inference(
    config.model.test_ds.file,
    output_prediction_file=output_prediction_file,
    output_nbest_file=output_nbest_file,
    num_samples=10, # setting to -1 will use all samples for inference
)
print("\n")
for question_id in all_preds:
    print(all_preds[question_id])

[NeMo I 2023-04-10 10:17:53 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpn8o8k41f/20a176d89bfd4f83a8d59dd2240c63ca_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-04-10 10:17:54 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-04-10 10:17:54 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    file: data_dir/squad/v2.0/train-v2.0.json
    batch_size: 8
    shuffle: true
    num_samples: 5000
    num_workers: 1
    drop_last: false
    pin_memory: false
    
[NeMo W 2023-04-10 10:17:54 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    file: data_dir/squad/v2.0/dev-v2.0.json
    batch_size: 8
    shuffle: fa

[NeMo I 2023-04-10 10:17:58 save_restore_connector:249] Model BERTQAModel was successfully restored from /content/work_dir/checkpoints/bert_squad_v2_0.nemo.


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit None Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


[NeMo I 2023-04-10 10:17:58 exp_manager:374] Experiments will be logged at work_dir/QA-SQuAD2/2023-04-10_10-01-14
[NeMo I 2023-04-10 10:17:58 exp_manager:797] TensorboardLogger has been set up


100%|██████████| 10/10 [00:00<00:00, 9984.06it/s]



France
10th and 11th centuries
Denmark, Iceland and Norway
Rollo
10th century
The Normans
Normandy
Rollo
10th century
William the Conqueror
